In [0]:
!curl ifconfig.co

35.229.201.223


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls /content/gdrive/My\ Drive/
!cp -r /content/gdrive/My\ Drive/BO-* /content/gdrive/My\ Drive/resume
!cd gdrive/My\ Drive/colab
!unzip /content/gdrive/My\ Drive/colab/ml-100k.zip
!ls gdrive/My\ Drive/colab/ml-100k/

 BO-DA-رزومه‌ها-E0		  nimac.gsite
 colab				  resume
'Getting started.pdf'		  SahabX-Sokan
'hadoop meeting feedback.gform'  'اعضای دانشکده.xlsx'
'Hadoop Meeting.gslides'	 'پروژه هوش سحاب.gdoc'
 NimaBehrang.pdf		 'مشخصات سرور.gsheet'
Archive:  /content/gdrive/My Drive/colab/ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data           bad CRC d354e598  (should be 55baee78)
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  infla

In [2]:
!pip install -q tensorflow-gpu==2.0.0a0
!pip install -q tensorboard
!pip install -q tensorboardcolab

     |████████████████████████████████| 332.1MB 53kB/s 
     |████████████████████████████████| 419kB 47.0MB/s 
     |████████████████████████████████| 3.0MB 47.1MB/s 


In [3]:
from tensorflow.python import keras
import tensorflow as tf
from tensorboardcolab import *
tbc=TensorBoardColab(port=6006, graph_path='/tmp/logs', startup_waiting_time=8)

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://a8d6fbf7.ngrok.io


In [0]:
# input_user = keras.Input(shape=(1,))  # Returns a placeholder tensor
# input_item = keras.Input(shape=(1,))  # Returns a placeholder tensor
# embed_user = keras.layers.Embedding(2000, 64)(input_user)
# embed_item = keras.layers.Embedding(2000, 64)(input_item)
# multiply = keras.layers.Multiply()([embed_user, embed_item])
# rate = keras.layers.add(tf.unstack(multiply, axis=2))
# model = keras.Model(inputs=[input_user, input_item], outputs = keras.activations.sigmoid(rate))
# model.summary()

In [5]:
input_user = keras.Input(shape=(1,))  # Returns a placeholder tensor
input_item = keras.Input(shape=(1,))  # Returns a placeholder tensor
embed_user = keras.layers.Embedding(2000, 64)(input_user)
embed_item = keras.layers.Embedding(2000, 64)(input_item)
dot = keras.layers.dot(inputs=[embed_user, embed_item],axes=2)
model_opt = keras.Model(inputs=[input_user, input_item], outputs = keras.activations.sigmoid(dot))
model_opt.summary()


sgd = tf.keras.optimizers.SGD(lr=0.01, clipvalue=0.5)
adam = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.999, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
nadam = tf.keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

model_opt.compile(optimizer='adam',
            loss='mean_squared_error',
            metrics=['accuracy','mae'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 64)        128000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 64)        128000      input_2[0][0]                    
______________________________________________________________________________________________

In [37]:
import numpy as np
data = np.genfromtxt("/content/gdrive/My Drive/colab/ml-100k/u.data", dtype=int)
train_data = []
test_data = []
UPL=50
topK=10
users = np.unique(data[:,0])
for user in users:
  userrows = data[(np.where(data[:,0] == user)),:][0]
  if len(userrows) > UPL + topK:
    np.random.shuffle(userrows)
    for item in userrows[:UPL]:
      train_data.append(item)
    for item in userrows[UPL:]:
      test_data.append(item)
train_data = np.asarray(train_data)
test_data = np.asarray(test_data)

print(np.shape(train_data))
print(np.shape(test_data))
print(train_data[0])
print(test_data[0])


(24700, 4)
(59716, 4)
[        1        52         4 875072205]
[        1       105         2 875240739]


In [0]:
keras.backend.clear_session()

In [39]:
def castRate(inp):
  return inp/5-0.1

x_train = [train_data[:,0],train_data[:,1]]
y_train = [castRate(x) for x in train_data[:,2]]
x_test = [test_data[:,0],test_data[:,1]]
y_test = [castRate(x) for x in test_data[:,2]]

logdir="/tmp/logs/2"
checkpoint_path = logdir + "/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=checkpoint_path, histogram_freq=0,
                                             write_graph=True, write_grads=False, write_images=False,
                                             embeddings_freq=0, embeddings_layer_names=None,
                                             embeddings_metadata=None, embeddings_data=[], update_freq='epoch')
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=1, verbose=1,
                                               mode='auto', baseline=None, restore_best_weights=False)

fit = model_opt.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10,
                    callbacks = [cp_callback, tb_callback], batch_size=500)
                # callbacks = [cp_callback, tb_callback, es_callback], batch_size=100)

W0729 15:11:36.366191 140106739226496 tf_logging.py:161] Model failed to serialize as JSON. Ignoring... ('Not JSON Serializable:', b'\n\x07Sigmoid\x12\x07Sigmoid\x1a\ndot/MatMul*\x07\n\x01T\x12\x020\x01')


Train on 24700 samples, validate on 59716 samples
Epoch 1/10
24500/24700 [============================>.] - ETA: 0s - loss: 0.0507 - accuracy: 0.0000e+00 - mae: 0.1876
Epoch 00001: saving model to /tmp/logs/2/cp.ckpt
24700/24700 [==============================] - 1s 22us/sample - loss: 0.0507 - accuracy: 0.0000e+00 - mae: 0.1876 - val_loss: 0.0511 - val_accuracy: 0.0000e+00 - val_mae: 0.1884
Epoch 2/10
24500/24700 [============================>.] - ETA: 0s - loss: 0.0506 - accuracy: 0.0000e+00 - mae: 0.1883
Epoch 00002: saving model to /tmp/logs/2/cp.ckpt
24700/24700 [==============================] - 1s 22us/sample - loss: 0.0506 - accuracy: 0.0000e+00 - mae: 0.1883 - val_loss: 0.0510 - val_accuracy: 0.0000e+00 - val_mae: 0.1885
Epoch 3/10
19500/24700 [======================>.......] - ETA: 0s - loss: 0.0508 - accuracy: 0.0000e+00 - mae: 0.1887
Epoch 00003: saving model to /tmp/logs/2/cp.ckpt
24700/24700 [==============================] - 1s 21us/sample - loss: 0.0506 - accuracy: 0.00

In [40]:
def calcHitRatio(test_data):
  x_test = [test_data[:,0],test_data[:,1]]
  y_test = [castRate(x) for x in test_data[:,2]]
  predict_rate = model_opt.predict(x_test)
  users_number = np.unique(test_data[:,0])
  predict = [(user, movie, rate[0][0]) for user,movie,rate in zip(test_data[:,0], test_data[:,1],predict_rate)]
  real = [(user, movie, castRate(rate)) for user,movie,rate in zip(test_data[:,0], test_data[:,1], test_data[:,2])]

  print(predict[:20])
  print(real[:20])

  avr, num = (0,0)
  for selected_user in users_number:
    predict_for_user = sorted([(user, movie, rate) for user,movie,rate in predict if user == selected_user], key=lambda x: (x[2], x[1]), reverse=True)
    real_for_user = sorted([(user, movie, rate) for user,movie,rate in real if user == selected_user], key=lambda x: (x[2], x[1]), reverse=True)
    count = 0
    if selected_user == 1:
        print(predict_for_user)
        print(real_for_user)
    for u,m,r in real_for_user[:topK]:
      count += len([x for x,y,z in predict_for_user[:topK] if y == m])
    if selected_user < 50:
      print(count)
    avr = (avr*num + count)/(num+1)
    num = num+1
  return (avr,num)

print(calcHitRatio(test_data))

[(1, 105, 0.60632575), (1, 244, 0.6088274), (1, 117, 0.61653256), (1, 258, 0.61394703), (1, 232, 0.6118123), (1, 35, 0.6093758), (1, 8, 0.6131339), (1, 192, 0.6094094), (1, 38, 0.6144154), (1, 20, 0.61947894), (1, 25, 0.61220956), (1, 239, 0.61285853), (1, 77, 0.6102416), (1, 87, 0.61466575), (1, 227, 0.60791177), (1, 134, 0.6135156), (1, 177, 0.6166001), (1, 193, 0.6131902), (1, 121, 0.61141425), (1, 102, 0.611959)]
[(1, 105, 0.30000000000000004), (1, 244, 0.30000000000000004), (1, 117, 0.5), (1, 258, 0.9), (1, 232, 0.5), (1, 35, 0.1), (1, 8, 0.1), (1, 192, 0.7000000000000001), (1, 38, 0.5), (1, 20, 0.7000000000000001), (1, 25, 0.7000000000000001), (1, 239, 0.7000000000000001), (1, 77, 0.7000000000000001), (1, 87, 0.9), (1, 227, 0.7000000000000001), (1, 134, 0.7000000000000001), (1, 177, 0.9), (1, 193, 0.7000000000000001), (1, 121, 0.7000000000000001), (1, 102, 0.30000000000000004)]
[(1, 110, 0.62027943), (1, 139, 0.61965835), (1, 20, 0.61947894), (1, 124, 0.6193177), (1, 2, 0.6185996